DP를 활용하여 RL을 수행하는 방식은, RL이 학습되는 공간에 대해 전이확률, 보상 등의 모든 정보가 주어진 MDP 환경이다.

DP를 구현하기 위해서는 RL과정을 bellman equation에 따라 크게 두 가지로 나누는데,

* action을 취함으로써 얻게 되는 현재의 보상
* action 이후의 state들로 얻게되는 보상의 할인된 가치

이렇게 나뉘는 구조는 DP로 구현하기에 용이하기 때문에 DP를 활용한다.

DP를 활용한 방식에는 Policy Iteration 방식과 Value Iteration 방식이 있다.

## Policy Iteration

<img src="../lectures/imgs/ch3_7.jpg"/>

Policy Iteration은 Policy Evaluation과 Policy Improvement 두 가지 절차로 진행된다.

Policy Evaluation은 가장 초기에 주어지는 policy로 각 state에 대해 최적의 value function을 찾아 평가하는 과정이다.

Policy Improvement는 Evaluation으로 찾은 최적의 value function 값으로 다음 state로 greedy하게 진행하여 policy를 update하는 과정이다.

이 두 과정을 반복하여 Optimal Value와 Optimal Policy를 찾아낸다.

In [1]:
import numpy as np

In [2]:
def get_state(state, action):
    
    # 상, 하, 좌, 우
    action_grid = [(-1,0),(1,0),(0,-1),(0,1)]
    
    state[0] += action_grid[action][0]
    state[1] += action_grid[action][1]
    
    # bouce back to edge
    if state[0] < 0:
        state[0] = 0
    elif state[0] > 3:
        state[0] = 3
        
    if state[1] < 0:
        state[1] = 0
    elif state[1] > 3:
        state[1] = 3
        
    return state[0], state[1]

In [3]:
grid_width = 4
grid_height = grid_width
action = [0,1,2,3]
# why add one more space for action?
policy = np.empty([grid_height, grid_width, len(action)], dtype=float)

for i in range(grid_height):
    for j in range(grid_width):
        for k in range(len(action)):
            if i == j and ((i==0) or (i==3)):
                policy[i][j] = 0.00
            else:
                policy[i][j] = 0.25

In [4]:
policy

array([[[0.  , 0.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 0.  , 0.  ]]])

In [5]:
iter_num = 10
verbose = True
reward = -1
dis = 0.9

In [6]:
## poilcy evaluation

# value = policy_evaluation(grid_width, grid_height, action, policy, 100)

post_value_table = np.zeros([grid_height, grid_width],dtype=float)

if iter_num == 0:
    print("result is post_value_table")
    
else:
    for iteration in (range(iter_num)):
        next_value_table = np.zeros([grid_height, grid_width],dtype=float)
        for i in range(grid_height):
            for j in range(grid_width):
                if i==j and ((i==0) or (i==3)):
                    value_t = 0
                else:
                    value_t = 0
                    for act in action:
                        i_, j_ = get_state([i,j], act)
                        value = policy[i][j][act] * (reward + dis*post_value_table[i_][j_])
                        value_t += value
                
                next_value_table[i][j] = round(value_t, 3)
        
        if verbose:
            print("iteration:{} \n{}\n".format(iteration, next_value_table))
            
        post_value_table = next_value_table                

iteration:0 
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]

iteration:1 
[[ 0.    -1.675 -1.9   -1.9  ]
 [-1.675 -1.9   -1.9   -1.9  ]
 [-1.9   -1.9   -1.9   -1.675]
 [-1.9   -1.9   -1.675  0.   ]]

iteration:2 
[[ 0.    -2.232 -2.659 -2.71 ]
 [-2.232 -2.609 -2.71  -2.659]
 [-2.659 -2.71  -2.609 -2.232]
 [-2.71  -2.659 -2.232  0.   ]]

iteration:3 
[[ 0.    -2.688 -3.32  -3.416]
 [-2.688 -3.224 -3.371 -3.32 ]
 [-3.32  -3.371 -3.224 -2.688]
 [-3.416 -3.32  -2.688  0.   ]]

iteration:4 
[[ 0.    -3.077 -3.879 -4.031]
 [-3.077 -3.727 -3.945 -3.879]
 [-3.879 -3.945 -3.727 -3.077]
 [-4.031 -3.879 -3.077  0.   ]]

iteration:5 
[[ 0.    -3.404 -4.36  -4.56 ]
 [-3.404 -4.16  -4.423 -4.36 ]
 [-4.36  -4.423 -4.16  -3.404]
 [-4.56  -4.36  -3.404  0.   ]]

iteration:6 
[[ 0.    -3.683 -4.768 -5.014]
 [-3.683 -4.522 -4.834 -4.768]
 [-4.768 -4.834 -4.522 -3.683]
 [-5.014 -4.768 -3.683  0.   ]]

iteration:7 
[[ 0.    -3.919 -5.117 -5.402]
 [-3.919 -4.833 -5.18  -5.117]


In [7]:
value = post_value_table

## policy improvement

action_match = ['Up', 'Down', 'Left', 'Right']
action_table = []

for i in range(grid_height):
    for j in range(grid_width):
        q_func_list = []
        if i == j and ((i==0) or (i==3)):
            action_table.append("T")
        else:
            for k in range(len(action)):
                i_, j_ = get_state([i,j],k)
                q_func_list.append(value[i_][j_])
            max_actions = [action_v for action_v, x in enumerate(q_func_list) if x == max(q_func_list)] 
            
            policy[i][j] = [0] * len(action)
            for y in max_actions:
                policy[i][j][y] = (1/len(max_actions))
            
            idx = np.argmax(policy[i][j])
            action_table.append(action_match[idx])
            
#     break
action_table=np.asarray(action_table).reshape((grid_height, grid_width))            

In [8]:
action_table

array([['T', 'Left', 'Left', 'Down'],
       ['Up', 'Up', 'Down', 'Down'],
       ['Up', 'Up', 'Down', 'Down'],
       ['Up', 'Right', 'Right', 'T']], dtype='<U5')

## Value Iteration

<img src="../lectures/imgs/ch3_10.jpg"/>

Value Iteration은 두 가지 절차로 진행되는 Policy Iteration과 달리 한 번 각 state에 대해 optimal 한 move를 함.

In [9]:
## value evaluation

iter_num = 10
verbose = True
reward = -1
dis = 0.9

# value = policy_evaluation(grid_width, grid_height, action, policy, 100)

post_value_table = np.zeros([grid_height, grid_width],dtype=float)

action_match = ['Up', 'Down', 'Left', 'Right']

if iter_num == 0:
    print("result is post_value_table")
    
else:
    for iteration in (range(iter_num)):
        next_value_table = np.zeros([grid_height, grid_width],dtype=float)
        action_table = list()
        for i in range(grid_height):
            action_row_table = list()
            for j in range(grid_width):
                if i==j and ((i==0) or (i==3)):
                    value_t = 0
                    action_t = "T"
                else:
                    value_t_list = list()
                    for act in action:
                        i_, j_ = get_state([i,j], act)
                        value = reward + dis*post_value_table[i_][j_]
                        value_t_list.append(value)
                
                    value_t = max(value_t_list)
                    action_t = action_match[np.argmax(value_t_list)]
                
                action_row_table.append(action_t)
                next_value_table[i][j] = value_t 
            
            action_table.append(action_row_table)
        
        if verbose:
            print("iteration:{} \n{}\n".format(iteration, next_value_table))
            print("iteration:{} \n{}\n".format(iteration, action_table))
        
        
        post_value_table = next_value_table                

iteration:0 
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]

iteration:0 
[['T', 'Up', 'Up', 'Up'], ['Up', 'Up', 'Up', 'Up'], ['Up', 'Up', 'Up', 'Up'], ['Up', 'Up', 'Up', 'T']]

iteration:1 
[[ 0.  -1.  -1.9 -1.9]
 [-1.  -1.9 -1.9 -1.9]
 [-1.9 -1.9 -1.9 -1. ]
 [-1.9 -1.9 -1.   0. ]]

iteration:1 
[['T', 'Left', 'Up', 'Up'], ['Up', 'Up', 'Up', 'Up'], ['Up', 'Up', 'Up', 'Down'], ['Up', 'Up', 'Right', 'T']]

iteration:2 
[[ 0.   -1.   -1.9  -2.71]
 [-1.   -1.9  -2.71 -1.9 ]
 [-1.9  -2.71 -1.9  -1.  ]
 [-2.71 -1.9  -1.    0.  ]]

iteration:2 
[['T', 'Left', 'Left', 'Up'], ['Up', 'Up', 'Up', 'Down'], ['Up', 'Up', 'Down', 'Down'], ['Up', 'Right', 'Right', 'T']]

iteration:3 
[[ 0.   -1.   -1.9  -2.71]
 [-1.   -1.9  -2.71 -1.9 ]
 [-1.9  -2.71 -1.9  -1.  ]
 [-2.71 -1.9  -1.    0.  ]]

iteration:3 
[['T', 'Left', 'Left', 'Down'], ['Up', 'Up', 'Up', 'Down'], ['Up', 'Up', 'Down', 'Down'], ['Up', 'Right', 'Right', 'T']]

iteration:4 
[[ 0.   -1.   -1.9  -2.71]
 [-1.  